In [1]:
from nnsight import LanguageModel

model = LanguageModel("openai-community/gpt2-xl", device_map="cuda:0", dispatch=True)

/share/u/caden/.conda/envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/share/u/caden/.conda/envs/test/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
correct_token = model.tokenizer.encode(" Paris")
results = []

def decoder(x):
    return model.lm_head(model.transformer.ln_f(x))

with model.trace("The capital of France is"):
    for layer in model.transformer.h:

        logits = decoder(layer.output[0])
        tokens = logits.softmax(-1)[:,-1,correct_token]
        results.append(tokens.save())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# results = [r.value.item() for r in results]

plt.plot(results)
plt.ylim(0, 1)
plt.xlabel('Layer')  # Replace with your desired label
plt.ylabel('P( Paris)')  # Replace with your desired label
plt.show()

In [ ]:
from tqdm import tqdm

In [4]:
prompt = "The Space Needle is in downtown"
prompt = model.tokenizer.encode(prompt)

correct_token = model.tokenizer.encode(" Seattle")

low_memory_rome_causal_tracing(model, prompt, correct_token)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 

: 

In [3]:
def low_memory_rome_causal_tracing(model, prompt, target):

    clean_acts = []
    n_layers = len(model.transformer.h)
    n_tokens = len(prompt)
    
    with model.trace(prompt):
        for i, layer in enumerate(model.transformer.h):
            clean_acts.append(layer.output[0].save())

        probs = model.lm_head.output.softmax(-1)
        clean_value = probs[:,-1, target]
        clean_value.save()

    results = []
    clean_value = clean_value.value.item()

    for t in range(n_tokens):

        per_token_result = []
        for layer in range(n_layers):
            with model.trace(prompt, scan=False):
                # Corrupt the subject tokens
                model.transformer.wte.output[:,0:3,:][:] = 0.

                # Restore the clean activations
                model.transformer.h[layer].output[0][:,t,:] = clean_acts[layer][:,t,:]
                
                probs = model.lm_head.output.softmax(-1)
                difference = clean_value - probs[:,-1, target]
                
                per_token_result.append(difference.item().save())                   

        results.append(per_token_result)

    return results

In [ ]:
prompt = "The Space Needle is in downtown"
prompt = model.tokenizer.encode(prompt)

correct_token = model.tokenizer.encode(" Seattle")

n_tokens = len(prompt)
n_layers = len(model.transformer.h)

with model.trace(remote=True, validate=False) as tracer:

    clean_acts = []

    with tracer.invoke(prompt):
        for i, layer in enumerate(model.transformer.h):
            clean_acts.append(layer.output[0])

        probs = model.lm_head.output.softmax(-1)
        clean_value = probs[:,-1, correct_token]

    results = []

    for t in range(n_tokens):

        per_token_result = []
        for layer in range(n_layers):
            with tracer.invoke(prompt, scan=False):
                # Corrupt the subject tokens
                model.transformer.wte.output[:,0:3,:][:] = 0.

                # Restore the clean activations
                model.transformer.h[layer].output[0][:,t,:] = clean_acts[layer][:,t,:]
                
                probs = model.lm_head.output.softmax(-1)
                difference = clean_value - probs[:,-1, correct_token]
                
                per_token_result.append(difference.item().save())                   

        results.append(per_token_result)

In [ ]:
new_results=  []

for i in results:
    row = []
    for j in i:
        temp = j.value
        row.append(temp)
    new_results.append(row)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


y_labels = ["The*", "Space*", "Need*", "le*", "is", "in", "downtown"]

fig, ax = plt.subplots()

cax = ax.imshow(new_results, cmap='Purples_r', aspect='auto')

# Set the y-axis labels
ax.set_yticks(np.arange(len(y_labels)))
ax.set_yticklabels(y_labels)

# Set the labels for the axes
ax.set_xlabel('single restored layer within GPT-2-XL')

cbar = fig.colorbar(cax, ax=ax, orientation='vertical')
plt.show()

In [ ]:
with model.trace() as tracer:

    for i in range(12):
        with tracer.invoke("hello"):
            if i == 11:
                test = model.lm_head.output[:,-1,:].save()

In [ ]:
with model.trace("input", remote=True):
    output = model.output.save()

In [ ]:
with model.trace() as tracer:
    with tracer.invoke("Prompt One"):
        pass
    with tracer.invoke("Prompt Two"):
        pass

In [ ]:
from nnsight import LanguageModel

model = LanguageModel("openai-community/gpt2", device_map="cuda:0", dispatch=True)

In [ ]:
with model.trace("hello"):
    print(model.transformer.h[0].mlp.input.shape)
    print(model.transformer.h[0].mlp.output.shape)

In [ ]:
import torch.nn as nn
import torch
from torch.utils.data import DataLoader

n_tokens = 10
answer = " Paris"
answer_token = model.tokenizer.encode(answer)[0]

class LORA(nn.Module):
    def __init__(self, module, dim, r: int) -> None:
        super(LORA, self).__init__()
        self.r = r
        self.module = module

        self.WA = torch.nn.Parameter(torch.randn(dim, self.r), requires_grad=True)
        self.WB = torch.nn.Parameter(torch.zeros(self.r, dim), requires_grad=True)

    def __call__(self, alpha:float=1.0):
        A_x = torch.matmul(self.module.input[0][0], self.WA)
        BA_x = torch.matmul(A_x, self.WB)
        h = BA_x + self.module.output

        self.module.output = h * alpha

    def parameters(self):
        return [self.WA, self.WB]

lora = LORA(model.transformer.h[0].mlp, 768, 4).to("cuda:0")

In [ ]:
dataset = [["_" * n_tokens, answer_token]] * 100
dataloader = DataLoader(dataset, batch_size=10)

optimizer = torch.optim.AdamW(lora.parameters(), lr=.1)
loss_fn = nn.CrossEntropyLoss()

for i, (inputs, targets) in enumerate(dataloader):
    optimizer.zero_grad()

    with model.trace(inputs) as runner:
        lora()

        logits = model.lm_head.output.save()

    loss = loss_fn(logits[:, -1], targets.to("cuda:0"))
    loss.backward()

    optimizer.step()

In [ ]:
with model.generate(dataset[0][0]) as generator:
    lora()

    out = model.generator.output.save()



print(model.tokenizer.batch_decode(out))

with model.generate(dataset[0][0]) as generator:

    out = model.generator.output.save()

print(model.tokenizer.batch_decode(out))

In [ ]:
from nnsight import LanguageModel
name = "meta-llama/Meta-Llama-3-70B"
model = LanguageModel(name)
with model.trace("input"):
    pass